In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup, SoupStrainer
from selenium import webdriver

In [ ]:
driver = webdriver.Chrome()

In [ ]:
base_url = 'https://courses.illinois.edu'
driver.get('https://courses.illinois.edu/schedule/2025/spring')
subjects_html = driver.page_source
subjects_soup = BeautifulSoup(subjects_html, 'html', parse_only = SoupStrainer('table'))
subject_urls = [subject.find('a')['href'] for subject in subjects_soup.find_all('tr', role = 'row')[1:]]
subject_urls

all_class_rows = []

for subject_url in subject_urls:
    
    driver.get(base_url + subject_url)
    classes_html = driver.page_source
    classes_soup = BeautifulSoup(classes_html, 'html', parse_only = SoupStrainer('table'))
    class_urls = [class_.find('a')['href'] for class_ in classes_soup.find_all('tr', role = 'row')[1:]]
    
    for class_url in class_urls:

        class_name = ' '.join(class_url.split('/')[-2:])
        driver.get(base_url + class_url)
        class_html = driver.page_source
        class_soup = BeautifulSoup(class_html, 'html', parse_only = SoupStrainer('table'))
        
        rows = class_soup.find_all('tr', {'role' : 'row'})
        rows = [row.find_all('div') for row in rows[1:]]
        rows = [[elem.get_text(' ').strip() for elem in row] for row in rows]

        for row in rows:
            sections = len(row) // 6
            if sections == 1:
                all_class_rows.append([class_name] + row)
                continue
            all_class_rows += [[class_name] + row[i::sections] for i in range(sections)]

In [ ]:
headings = ['Class', 'Type', 'Section', 'Time', 'Day', 'Location', 'Instructor']
uiuc_courses_df = pd.DataFrame(all_class_rows, columns = headings)
uiuc_courses_df.to_csv('SP25_UIUC_courses.csv', index = False)
uiuc_courses_df

In [8]:
df = pd.read_csv('SP25_UIUC_courses.csv')
df2 = df.loc[:, 'Location']

building = 'Loomis'
room = '144'
idxs = []
locs = {}

for idx, location in enumerate(df2):
    location_list = location.split()
    if len(location_list) > 1:
        if location_list[0] == room and location_list[1] == building:
            idxs.append(idx)

classes_df = pd.DataFrame(df.iloc[idxs])
classes_df

,Class,Type,Section,Time,Day,Location,Instructor
517,AE 443,Lecture-Discussion,A1,02:00PM - 03:20PM,TR,144 Loomis Laboratory,"Merret, J"
4051,ECE 489,Lecture,AL,09:30AM - 10:50AM,TR,144 Loomis Laboratory,"Yim, J"
7632,MATH 103,Lecture,AL1,11:00AM - 12:20PM,TR,144 Loomis Laboratory,"Pinto, A"
8398,ME 446,Lecture,AL,09:30AM - 10:50AM,TR,144 Loomis Laboratory,"Yim, J"
10205,PHYS 326,Lecture,A,12:30PM - 01:50PM,TR,144 Loomis Laboratory,"Golding, I"
10230,PHYS 427,Lecture,A,03:30PM - 04:50PM,TR,144 Loomis Laboratory,"Draper, P"
10234,PHYS 435,Lecture,A,10:00AM - 10:50AM,MWF,144 Loomis Laboratory,"Eckstein, J"
10239,PHYS 436,Lecture,A,09:00AM - 09:50AM,MWF,144 Loomis Laboratory,"Kou, A"
10979,SE 422,Lecture,AL,09:30AM - 10:50AM,TR,144 Loomis Laboratory,"Yim, J"
11143,SOC 101,Lecture-Discussion,A,01:00PM - 01:50PM,MW,144 Loomis Laboratory,"Vogler, S"
